In [7]:
import networkx as nx
import random
from ortools.linear_solver import pywraplp

# Configurações gerais
n = 100  # número de vértices
p = 0.1  # probabilidade de aresta
random.seed(42)

# Gerando o grafo aleatório
G = nx.gnp_random_graph(n, p)
for (u, v) in G.edges():
    G.edges[u, v]['weight'] = random.randint(10, 500)

# Visualização básica (opcional)
print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())


Número de vértices: 100
Número de arestas: 474


In [8]:
import itertools

def solve_with_subtour_elimination(G):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variáveis de decisão
    x = {}
    for u, v in G.edges():
        x[u, v] = solver.BoolVar(f'x_{u}_{v}')

    # Função objetivo
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * x[u, v] for u, v in G.edges()))

    # Restrição de árvore geradora
    for v in G.nodes():
        solver.Add(solver.Sum(x[u, v] for u, v in G.edges() if u == v or v == v) == 1)

    # Eliminação de ciclos (SEC)
    for subset in itertools.combinations(G.nodes(), 2):
        u, v = subset
        if u != v:
            solver.Add(solver.Sum(x[i, j] for i, j in nx.edge_boundary(G, {u}, {v})) <= len(subset) - 1)

    # Limite de tempo de execução
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return solver.Objective().Value()
    else:
        print('Nenhuma solução encontrada.')
        return None


In [11]:
def solve_with_mtz(G):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    n = G.number_of_nodes()
    c = n  # um número grande o suficiente

    # Variáveis de decisão
    y = {}
    u_labels = {}  # Alterando o nome para evitar colisão com a variável 'u'
    
    for node in G.nodes():
        u_labels[node] = solver.IntVar(0, c - 1, f'u_{node}')
        for neighbor in G.nodes():
            y[node, neighbor] = solver.BoolVar(f'y_{node}_{neighbor}')

    # Função objetivo
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * y[u, v] for u, v in G.edges()))

    # Restrição de árvore geradora direcionada
    for v in G.nodes():
        if v != 0:  # assumindo que o nó 0 é a raiz
            solver.Add(solver.Sum(y[u, v] for u, v in G.edges() if u == v or v == v) == 1)

    # Rótulos nos vértices (MTZ)
    for u, v in G.edges():
        solver.Add(u_labels[u] - u_labels[v] + c * y[u, v] <= c - 1)

    # Limite de tempo de execução
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return solver.Objective().Value()
    else:
        print('Nenhuma solução encontrada.')
        return None


In [49]:
import networkx as nx

def read_graph_from_file(file_path):
    with open(file_path, 'r') as f:
        # Primeira linha: n (vértices), nc (centrais), m (arestas)
        n, nc, m = map(int, f.readline().strip().split())
        
        # Próximas nc linhas: centrais e graus mínimos
        centrais = {}
        for _ in range(nc):
            i, d = map(int, f.readline().strip().split())
            centrais[i] = d
        
        # Próximas m linhas: arestas e custos
        G = nx.Graph()
        for _ in range(m):
            i, j, c = map(int, f.readline().strip().split())
            G.add_edge(i, j, weight=c)
        
    return G, centrais

# Exemplo de uso
file_path = './Instancias/tb8ch30_0.txt'
grafo, centrais = read_graph_from_file(file_path)

# Exibindo o grafo e as informações dos vértices centrais
print("Grafo:", grafo.edges(data=True))
print("Centrais e graus mínimos:", centrais)


Grafo: [(1, 2, {'weight': 396}), (1, 3, {'weight': 283}), (1, 4, {'weight': 198}), (1, 5, {'weight': 256}), (1, 6, {'weight': 268}), (1, 7, {'weight': 260}), (1, 8, {'weight': 290}), (1, 9, {'weight': 228}), (1, 10, {'weight': 109}), (1, 11, {'weight': 257}), (1, 12, {'weight': 168}), (1, 13, {'weight': 170}), (1, 14, {'weight': 271}), (1, 15, {'weight': 343}), (1, 16, {'weight': 113}), (1, 17, {'weight': 37}), (1, 18, {'weight': 233}), (1, 19, {'weight': 299}), (1, 20, {'weight': 324}), (1, 21, {'weight': 351}), (1, 22, {'weight': 257}), (1, 23, {'weight': 126}), (1, 24, {'weight': 302}), (1, 25, {'weight': 255}), (1, 26, {'weight': 101}), (1, 27, {'weight': 172}), (1, 28, {'weight': 198}), (1, 29, {'weight': 179}), (1, 30, {'weight': 282}), (1, 31, {'weight': 184}), (1, 32, {'weight': 23}), (1, 33, {'weight': 286}), (1, 34, {'weight': 150}), (1, 35, {'weight': 316}), (1, 36, {'weight': 248}), (1, 37, {'weight': 305}), (1, 38, {'weight': 122}), (1, 39, {'weight': 306}), (1, 40, {'weig

In [50]:
import time

def compare_formulations(G):
    start_time = time.time()
    obj_value_subtour = solve_with_subtour_elimination(G)
    time_subtour = time.time() - start_time

    start_time = time.time()
    obj_value_mtz = solve_with_mtz(G)
    time_mtz = time.time() - start_time

    print(f"Subtour Elimination: Objective Value = {obj_value_subtour}, Time = {time_subtour} seconds")
    print(f"MTZ: Objective Value = {obj_value_mtz}, Time = {time_mtz} seconds")

# Exemplo de comparação
compare_formulations(grafo)


Solução ótima encontrada!
Solução ótima encontrada!
Subtour Elimination: Objective Value = 6.0, Time = 0.23805499076843262 seconds
MTZ: Objective Value = 6.0, Time = 0.25995349884033203 seconds
